# Team - LogisticRegression
Tobias Vogt, Michael Lappert, Tobias Bischof


# 1. Introduction
As part of the "Data Science" module, the class was given the task of making a prediction for the price of used cars. The aim is to make as accurate a prediction as possible for the price in a Kaggle competition. The participants should learn how to approach such a project by means of a practical example. To apply the theoretical material from the class. This work includes a Jupyter notebook and a presentation of the results. The work will be solved in groups of three or four. This work is limited to the train and a test file of the corresponding Kaggle competition. However, numerous functions as well as commands from the internet were used.

# 2. Libraries
This chapter covers all libraries used in this work.



In [ ]:
#importing data wrangling
import pandas as pd
import numpy as np
from pandas.api.types import is_object_dtype, is_numeric_dtype
import re
import pickle

#importing data viz
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize

# to display visuals in the notebook
%matplotlib inline

#ml 
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.tree import plot_tree
from sklearn.base import clone 

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

sns.set(color_codes=True)
sns.set_palette(sns.color_palette("muted"))

# 3. Functions
Functions use throughout the notebook are defined here for a better reader experience afterward



In [ ]:
# Adopted from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/blob/master/notebooks/WineRatingPredictor-1.ipynb
def missing_values_table(df):
    '''
    Utility functi# 2) Libraries


on to calculate the missing values in absolute and percent values.
    '''
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent],
                              axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = (mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))

    # Print some summary information
    print("The selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

# from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/tree/master/notebooks
def plot_distribution(df, target, column_values, column_name):
    # funtion to print distribution of a continuous variable
    # for categorical data

    for value in column_values:
        subset = df[
            df[column_name] == value]
        g = sns.kdeplot(subset[target],
                        label=value,
                        linewidth=3)

    # set title, legends and labels
    plt.ylabel("Density",
               size=14)
    plt.xlabel("{}".format(target),
               size=14)
    plt.title("Distribution of {} per {}"
              .format(target, column_name),
              size=16)

    return g


# Adopted from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/blob/master/notebooks/WineRatingPredictor-1.ipynb
def plot_histogram(df, column, b=None):
    '''
    function to print histogram
    with mean and median
    using distplot
    '''

    # set the histogram, mean and median
    g = sns.distplot(df[column],
                     kde=False,
                     bins=b)
    plt.axvline(x=df[column].mean(),
                linewidth=3,
                color='g',
                label="mean",
                alpha=0.5)
    plt.axvline(x=df[column].median(),
                linewidth=3,
                color='y',
                label="median",
                alpha=0.5)

    # set title, legends and labels
    plt.xlabel("{}".format(column),
               size=14)
    plt.ylabel("Count",
               size=14)
    plt.title("Distribution of {}".format(column),
              size=16)
    plt.legend(["mean", "median"])

    return g


# From dslectures.core
def convert_strings_to_categories(df):
    """
    A utility function to convert all string columns to Categorical data type.
    """
    for col in df.columns:
        if is_object_dtype(df[col]):
            df[col] = df[col].astype("category")


# Adopted from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/blob/master/notebooks/WineRatingPredictor-1.ipynb
def filling_df(df):
    '''
    function converts strings to categories and
    categories to numbers
    and imputes for missing values the median.
    '''    
    # converting strings to categories
    convert_strings_to_categories(df)

    # Converting categories to numbers
    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    
    # Imputing the median to the missing values
    median = df.median()
    df.fillna(median, inplace=True)

    return df


# Function takes the datepart and creates new columns with information gained out of the date
# (originally from https://course18.fast.ai/lessonsml1/lesson1.html)
def add_datepart(df, fldname, drop=True):
    '''
    Extracts datepart into several new more useful features
    '''
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, 
                                     infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 
            'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 
            'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        

#Function of lesson05 of ds
def print_scores(fitted_model):
    """Generates RMSE and R^2 scores from fitted Random Forest model."""

    yhat_train = fitted_model.predict(X_train)
    R2_train = fitted_model.score(X_train, y_train)
    yhat_valid = fitted_model.predict(X_valid)
    R2_valid = fitted_model.score(X_valid, y_valid)

    scores = {
        "RMSE on train:": rmse(y_train, yhat_train),
        "R^2 on train:": R2_train,
        "RMSE on valid:": rmse(y_valid, yhat_valid),
        "R^2 on valid:": R2_valid,
    }
    if hasattr(fitted_model, "oob_score_"):
        scores["OOB R^2:"] = fitted_model.oob_score_

    for score_name, score_value in scores.items():
        print(score_name, round(score_value, 3))
        

#Function from dslecture.core
def rmse(y, yhat):
    """A utility function to calculate the Root Mean Square Error (RMSE).
    
    Args:
        y (array): Actual values for target.
        yhat (array): Predicted values for target.
        
    Returns:
        rmse (double): The RMSE.
    """
    return np.sqrt(mean_squared_error(y, yhat))

# https://lewtun.github.io/dslectures/lesson09_overfitting/
def plot_fitting_graph(x, metric_train, metric_valid, metric_name='metric', xlabel='x', yscale='linear'):
    plt.plot(x, metric_train, label='train')
    plt.plot(x, metric_valid, label='valid')
    plt.yscale(yscale)
    plt.title('Fitting graph')
    plt.ylabel(metric_name)
    plt.xlabel(xlabel)
    plt.legend(loc='best')
    plt.grid(True)

# 4. Get the data
Since we all run our notebooks locally. we have stored all data paths of the group members in our master document. This makes it easier to work in one document.



In [ ]:
train = pd.read_csv('your_path')
test = pd.read_csv('your_path')
sample_submission = pd.read_csv('your_path')
train.shape, test.shape

## 4.1 Overview

In [ ]:
#With these first comands we got a rough overview of the two data sets.

In [ ]:
train.shape

In [ ]:
print("There are {} rows and {} columns in the train dataset."
      .format(train.shape[0], train.shape[1]))

print("There are {} rows and {} columns in the test dataset."
      .format(test.shape[0], test.shape[1]))

# 5. Explore the Data


In [ ]:
# Creating a copy of the data for exploration.
train_ex = train.copy()
test_ex = test.copy()
train_ex.shape, test_ex.shape

## 5.1 Study each attribute and its characteristics:

In this chapter of the notebook is looked at the following points:
- Name
- Type (categorical, int/float, bounded/unbounded, text, structered, etc.)
- % of missing values
- Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
- Possibly useful for the task?
- Type of distribution (Gaussian, uniform, logarithmic, etc.)

In [ ]:
train_ex.info()

In [ ]:
test_ex.info()

In [ ]:
# Checking if every carId belongs to one entry and hence are unique
len(train_ex) == train_ex.carId.nunique()

In [ ]:
# Looking at the first five row of the dataset to get an impretion how the dataset looks like
train_ex.head()

In [ ]:
# Looking at 10 randomly chosen rows to get an further impression and to see if something strange appears.
train_ex.sample(10)

Some first explanations about the features and the target:
* **price:** Target value, price of the car
* **carId:** Every row and therfore every entry has his own ID
* **dateCrawled:** Date when data was downloaded
* **name:** Some description about the car added to the portal
* **seller:** Wheter the seller of the corresponding car is `privat` or `gewerblich`
* **offerType:** Wheter the seller is looking or offering for a car
* **abtest:** _not known yet_
* **vehicleType:** What kind of car os offered
* **yearOfRegistration:** When the car was registered first
* **gearbox:** Wheter the gearbox is `manuell`or `automatik`
* **powerPS:** Amount of horsepower of the car
* **model:** Which model is the car
* **kilometer:** How many kilometer the car is already driven
* **monthOfRegistration:** In which month of the Registrationyear the car was registered
* **fuelType:** Which type of fuel the car uses
* **brand:** Brand of the car
* **notRepairedDamage:** If the car has some damage at the time selling
* **dateCreated:** When the entry of the car was created on the portal
* **nrOfPictures:** Number of pictures of the car in the inserat
* **postalCode:** Postalcode of the city where the car is at
* **lastSeen:** Last date when was looked at the car before it was sold

In [ ]:
# The majority of the features in the dataset are strings. Both datasets have some missing values.
# The strings we have to convert into numerical values and the missing values we have to handle by either
# dropping or filling them to work with the datasets in the machine learning models.
train_ex.describe()

In [ ]:
# All the missing values are from string features.
missing_values_table(train_ex)

In [ ]:
# plots from lecture 2
# Plotting the percantage of missing values
percentage_null = (train_ex.isnull().sum()/len(train_ex)).sort_values(ascending=False)
percentage_null = percentage_null[percentage_null > 0]
sns.barplot(percentage_null.values, percentage_null.index, color='b')

## 5.2 Exploring the object features first

In [ ]:
# function from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/blob/master/notebooks/WineRatingPredictor-1.ipynb
# Exploring the number of unique label of the object features
object_columns = (train_ex
                 .select_dtypes(include='object')
                 .columns)

for column in object_columns:
    print("{} has {} unique values."
          .format(column,
                 train_ex[column]
                 .nunique()))

**High cardinality features (> 300 values):**

* dateCrawled
* name
* lastSeen

**Moderate cardinality features (> 30 values):**

* model
* brand
* dateCreated

**Low cardinality features (< 10 values):**

* seller
* offerType
* abtest
* vehicleType
* gearbox
* fuelType
* notRepairedDamage

There seem to be some identical name's. This seems strange because the name's can be typed in individually and it is expected that every person writes at least a slighty different description.
There are more fuelTypes than just the most common ones like Diesel or Benzin.
The dateCreated has a relatively small number compared to the number of entries in the dataset.

In the following it is looked at each object feature's unique values (excluding model and dateCreated).

In [ ]:
# Because it makes sense to use the price for some plots, it is looked first to this feature.
# Price reaches up to 2e9. This is really unrealistic. Checking some carselling pages the maximum price is set to 5e6.
# By far the majority of prices lay around 0. 
train_ex = train_ex.loc[(train_ex.price < 40000)]
train_ex = train_ex.loc[(train_ex.price > 0)]
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_ex, 'price');

In [ ]:
# The brands reach from cheaper brands as 'kia', 'hyundai' or 'daewoo' to exclusive brands as 'jaguar' or 'porsche'.
# There is also some aggregation of cars called 'sonstige_autos'
# Volkswagen is by far the most common brand in the dataset. audi, bmw, mercedes benz and opel have the second highest 
# entries more or less equally distributed
brands = (train_ex.brand.value_counts())

print(brands.index)

brands.plot(kind='bar', fontsize=15)
plt.title('Counts per brand', fontsize=25)
plt.ylabel('Counts', fontsize=20)
plt.show()

In [ ]:
freq1_brands = list(
    brands[
        brands.values > 4000]
    .index)

freq2_brands = list(
    brands[
        brands.values < 600]
    .index)

# It can be observed that the 10 most fequent brands have a similar distribution, whereas the fewest frequent brands
# show an inhomogen distribution. The fewest frequent brands have higher prices in general. Both plots show a
# 'right-skewed' distribution in tendency.

# distribution of the price per brand
figsize(20, 10)
plt.rcParams['font.size'] = 14

# plot points distribution for 10 most frequent brands
plt.subplot(1, 2, 1)
plot_distribution(train_ex, "price",
                  freq1_brands, "brand");

# plot points distribution for 11 fewest frequent brands
plt.subplot(1, 2, 2)
plot_distribution(train_ex, "price",
                  freq2_brands, "brand");

In [ ]:
# There are almost none of the entries 'gewerblich'.
sellers = train_ex.seller.value_counts()

print(sellers)

sellers.plot(kind='bar', fontsize=20)
plt.title('Counts per seller', fontsize=25)
plt.ylabel('Counts', fontsize=20)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# There are not just 'Gesuch' but also 'Angebot'. By far the most etries are 'Angebot'.
offerTypes = train_ex.offerType.value_counts()

print(offerTypes)

offerTypes.plot(kind='bar', fontsize=20)
plt.title('Counts per offerType', fontsize=25)
plt.ylabel('Counts', fontsize=20)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Some seem to look for a car and are willing to pay quite some money for the searched car.
freq_offerTypes = list(
    offerTypes[
        offerTypes.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_offerTypes, "offerType");

In [ ]:
abtests = train_ex.abtest.value_counts()

abtests.plot(kind='bar', fontsize=20)
plt.title('Counts of abtest', fontsize=25)
plt.ylabel('Counts', fontsize=20)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# It could not exactly found out what the abtest is. But looking at the price distributions of its to labels there
# seems to be no difference and therfore it was not longer looked at.
freq_abtests = list(
    abtests[
        abtests.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_abtests, "abtest");

In [ ]:
# Beside the expected vehicleTypes there are also some aggregated in 'andere'. By far the most frequent vehicleTypes
# are limousine, kleinwagen and kombi.
vehicleTypes = train_ex.vehicleType.value_counts()

vehicleTypes.plot(kind='bar', fontsize=20)
plt.title('Counts per vehicleType', fontsize=25)
plt.ylabel('Counts', fontsize=20)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# suv's seem to have the widest range of high prices whereas kleinwagen seem to be the cheapest ones.
freq_vehicleTypes = list(
    vehicleTypes[
        vehicleTypes.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_vehicleTypes, "vehicleType");

In [ ]:
# Most cars have a manuell gearbox.
gearboxes = train_ex.gearbox.value_counts()

gearboxes.plot(kind='bar', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.title('Counts per gearbox', fontsize=25);
plt.xticks(rotation=0)
plt.show()

In [ ]:
# It lookes like people are paying more for automatik gearboxed cars.
freq_gearboxes = list(
    gearboxes[
        gearboxes.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_gearboxes, "gearbox");

In [ ]:
# benzin and diesel seem to be the by far most common fuelTypes.
fuelTypes = train_ex.fuelType.value_counts()

print(fuelTypes);

train_ex.fuelType.value_counts().plot(kind='bar', fontsize=20)
plt.title('Counts per fuelType', fontsize=25);
plt.ylabel('Counts', fontsize=20)
plt.xlabel('fuelType', fontsize=20)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# cars with the fuelType hybrid seem to be the most ecpensive ones. Cars with the benzin and andere seem to be the
# cheapest ones. Benzin compared with diesel (the two by far most common fuelTypes) is cheaper.
freq_fuelTypes = list(
    fuelTypes[
        fuelTypes.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_fuelTypes, "fuelType");

In [ ]:
# Comparing the kilometers and the corresponding prices, it could be interpreted, that for cars with fuelType elektro,
# people pay less although they have less kilometer then hybrid.
train_ex.groupby('fuelType')['kilometer'].mean()

In [ ]:
train_ex.groupby('fuelType')['price'].mean()

In [ ]:
# It can be observed, that hybrid is more expensive (even the most expensive) than elektro and diesel is more
# expensive than benzin. 
train_ex.groupby('fuelType')['price'].mean().sort_values().plot(kind='bar', fontsize=20)
plt.title('Mean price per fuelType', fontsize=25);
plt.ylabel('Mean price', fontsize=20);
plt.xlabel('fuelType', fontsize=20)
plt.xticks(rotation=0)

In [ ]:
# There seem to be not many car which has a notRepairedDamage. But one could think, that the ones who do have one,
# rather do not post anything than declaring it.
notRepairedDamages = train_ex.notRepairedDamage.value_counts()

train_ex.notRepairedDamage.value_counts().plot(kind='bar', fontsize=20)
plt.title('Counts per notRepairedDamage', fontsize=25);
plt.ylabel('Counts', fontsize=20)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# As expected is less paid for cars with a notRepairedDamage.
freq_notRepairedDamages = list(
    notRepairedDamages[
        notRepairedDamages.values > 0]
    .index)

plot_distribution(train_ex, "price",
                  freq_notRepairedDamages, "notRepairedDamage");

In [ ]:
# The following names are the 10 most listed in the dataset.
names = train_ex.name.value_counts()

freq_names = list(
    names[
        names.values > 270]
    .index)

plot_distribution(train_ex, "price",
                  freq_names, "name");


In [ ]:
models = train_ex.model.value_counts()

freq1_models = list(
    models[
        models.values > 5000]
    .index)

freq2_models = list(
    models[
        models.values < 15]
    .index)

# It can be observed that the 10 most fequent models have a similar distribution, whereas the fewest frequent models
# show an inhomogen distribution. The fewest frequent models have higher prices in general.

# distribution of the price per model
figsize(20, 10)
plt.rcParams['font.size'] = 14

# plot points distribution for 10 most frequent models
plt.subplot(1, 2, 1)
plot_distribution(train_ex, "price",
                  freq1_models, "model");

# plot points distribution for 11 fewest frequent models
plt.subplot(1, 2, 2)
plot_distribution(train_ex, "price",
                  freq2_models, "model");

## 5.3 Exploring the numerical features
The values which are unrealistic are cleaned right away in this section.

In [ ]:
# powerPS has values up to 20000 which seems to be really unrealistic. The maximum powerPS is set to 500. A lot of 
# the cars seem to have 0 powerPS which seems strange and are filtered out. These values are partly from carpieces.
# The distribution seems to be ok.
train_ex = train_ex.loc[(train_ex.powerPS < 500)]
train_ex = train_ex.loc[(train_ex.powerPS > 0)]
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_ex, 'powerPS');

In [ ]:
# the yearOfRegistration reaches from 0 up to 10000 which is also unrealistic. The timehorizon is set from 1930 to 2017.
# In 2016 is the latest date of lastSeen in this dataset.
# The distribution seems to be quite ok.
train_ex = train_ex.loc[(train_ex.yearOfRegistration<2017)]
train_ex = train_ex.loc[(train_ex.yearOfRegistration>1930)]
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_ex, 'yearOfRegistration');

In [ ]:
# There seems to be a limitation of 150000 kilometer on the homepage input parameters where the cars are sold. Therefore
# it seems likely that there are cars that have more than 150000 kilometers. One has to consider this when looking
# at cars with kilometers of 150000. Means they could possibly have higher kilometervalues. There seems also to be a
# little peak at the down side of kilometer values.
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_ex, 'kilometer');

In [ ]:
# Comparing the prices of the cars with different kilometervalues, it seems that there is something going on with the 
# prices of the cars which have 5000 kilometer. Comparing the mean and median values reveals that there must be big
# differences in the prices because the mean is significantly higher than the median. One has to take this into 
# account in the further feature engineering part. It has to be said though, that the mean price is in every case
# higher than the median price in this case.

# distribution of the price per model
figsize(20, 10)
plt.rcParams['font.size'] = 14

# plot points distribution for 10 most frequent models
plt.subplot(1, 2, 1)
plt.title('Median price per kilometer', fontsize=20)
train_ex.groupby('kilometer')['price'].median().plot(kind='bar', fontsize=20);
plt.xticks(rotation=45)

# plot points distribution for 11 fewest frequent models
plt.subplot(1, 2, 2);
plt.title('Mean price per kilometer', fontsize=20);
train_ex.groupby('kilometer')['price'].mean().plot(kind='bar', fontsize=20);
plt.xticks(rotation=45);

In [ ]:
# It is assumed that the high prices of old cars are due to rare oldtimers. Which would be a reasonable explanation.
train_ex.groupby('yearOfRegistration')['price'].median().plot(kind='bar', fontsize=20);
plt.title('Mean price per yearOfRegistration', fontsize=25);
plt.ylabel('price', fontsize=20);
plt.xlabel('yearOfRegistration', fontsize=20);
plt.xticks(rotation=90);

In [ ]:
postalCodes = (train_ex.postalCode.value_counts())

freq_postalCodes = list(
    postalCodes[
        postalCodes.values > 160]
    .index)

# It can be observed, that the two most frequent postalcodes are 10115 and 65428. Consulting Google with this postalCodes
# of Germany, it can be found, that the first postalCode belongs to Berlin-Mitte. This is a 'trendy' and exclusive
# and therefore expensive city part of Berlin. It is assumed, at the people who live there also own more expensive cars.
# This can be observed in the datadistribution as well.
# The second postalCode is from Rüsselsheim am Main. It got famous because the automotive brand opel produced there in the
# beginnig their cars. Furthermore it belongs to Frankfurt, the 'money-city' of Germany. Also here it is assumed, that the
# people own more expensive cars. Also this can be verified by the data.
# The rest of the postalCodes show a similar 'right-skewed' distribution.

plot_distribution(train_ex, "price",
                  freq_postalCodes, "postalCode");

#### Conclusion of the exploratory part
As always mentioned in the plots descriptions, all of the numeric features have unrealistic values and many outliers. This has to be considered in the feature engineering part.

The ordinal features labels are all understandable and do not need any further preparation. But the data itself is not equally distributed. Also are some strange connections discovered as for example between kilometers and price: Low kilometer cars have also the lowest median and mean prices. This is not reasonable.

## 5.4 For supervised learning tasks, identify the target attribute(s).

Since we have in our train set the prices for each vehicle, this is a supervised learning method. The training data that you enter into the algorithm contains the desired solutions, which is called a labels. In our case the label is the price. Because the price is a numerical value, it is predicted by a regression based on a set of features like (kilometer, powerPS, brand,...).

# 6. Visualize the data.

In [ ]:
# plots from lesson 1
# Most of the cars have around 50 to 150 powerPS and a price of below 5000.
chart = sns.jointplot(
    'powerPS', 'price',
    data=train_ex,
    kind='hex');

In [ ]:
# from https://lewtun.github.io/dslectures/lesson02_exploratory-data-analysis/
# To get an deeper insight the kilometer an yearOfRegistration feature is included in a scatterplot. The setting of the 
# yearOfRegistration is done the same as in the exploratory part. In this chart in gets even more obvious how many
# entries have none powerPS or a price of 0. Most of these entries have 150000 kilometers. It seems that there is a tendency
# of cars with more kilometers have also more powerPS but also that the entries with lower kilometervalues have
# have higher prices.

fig = sns.scatterplot(
    x="powerPS",
    y="price",
    data=train_ex,
    alpha=0.4,
    hue="kilometer",
    palette="coolwarm",
    size=train_ex["yearOfRegistration"]
)

# place legend outside of figure
fig.legend(loc="center left", bbox_to_anchor=(1.01, 0.6), ncol=1);

In [ ]:
# from https://github.com/cereniyim/Wine-Rating-Predictor-ML-Model/blob/master/notebooks/WineRatingPredictor-1.ipynb
# The observation from the exploratory part is validated: The hybrid cars are the most expensive and have also the 
# highest single prices.

# set plot size and font size
figsize(14, 8)
plt.rcParams['font.size'] = 14

# violin plot to see descriptive statistics 
# and distribution
# per fuelType
f = sns.violinplot(data=train_ex,
                   x="fuelType",
                   y="price");

f.set_xticklabels(f.get_xticklabels(),
                  rotation=45);

plt.title("Points from Different Tasters",
          size=16);

In [ ]:
# use plt.subplots() to create multiple plots
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
# put one plot on axis ax0
sns.distplot(train_ex["powerPS"], kde=True, ax=ax0)
# put second plot on axis ax1
sns.scatterplot("brand", "powerPS", data=train_ex, ax=ax1)
# tight_layout() fixes spacing between plots
fig.tight_layout()

In [ ]:
# use plt.subplots() to create multiple plots
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
# put one plot on axis ax0
sns.distplot(train_ex["kilometer"], kde=True, ax=ax0)
# put second plot on axis ax1
sns.scatterplot("brand", "kilometer", data=train_ex, ax=ax1)
# tight_layout() fixes spacing between plots
fig.tight_layout()

In [ ]:
# use plt.subplots() to create multiple plots
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
# put one plot on axis ax0
sns.distplot(train_ex["kilometer"], kde=True, ax=ax0)
# put second plot on axis ax1
sns.scatterplot("fuelType", "kilometer", data=train_ex, ax=ax1)
# tight_layout() fixes spacing between plots
fig.tight_layout()

In [ ]:
# use plt.subplots() to create multiple plots
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(15
                                                        , 7))
# put one plot on axis ax0
sns.distplot(train_ex["kilometer"], kde=False, ax=ax0)
# put second plot on axis ax1
sns.scatterplot("yearOfRegistration", "brand", data=train_ex, ax=ax1)
# tight_layout() fixes spacing between plots
fig.tight_layout()

## 6.1 Study the correlations between attributes.

In [ ]:
# plots from lesson 1
# Comparing the different integer features to each other to find some correlations
# Shops a cap in kilometers at 150'000. It seems that one can not choose moore than this on the website.
attributes = ['price', 'kilometer', 'powerPS', 'yearOfRegistration', 'monthOfRegistration', 'postalCode']
sns.pairplot(train_ex[attributes].dropna());

In [ ]:
# plots from lesson 1
# Comparing the as relevant considered different original object features to each other to find some correlations
# Shops a cap in kilometers at 150'000. It seems that one can not choose moore than this on the website.
attributes = ['price', 'powerPS', 'abtest', 'vehicleType',
              'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage']
sns.pairplot(train_ex[attributes].dropna());

In [ ]:
# dateCrawled, abtest, seller, offerType, and monthOfRegistratin are dropped because they are not expexted to give
# further value to the model.
corrs = train_ex.corr()
corrs['price'].sort_values(ascending=False).sum(), corrs['price'].sort_values(ascending=False)

In [ ]:
# To explore the correlations of the categorical features as well, the df's NaN values are filled with the median for 
# a first try.
train_ex_filled = filling_df(train_ex)

In [ ]:
train_ex_filled.drop(['nrOfPictures'], axis=1, inplace=True); train_ex_filled.head()

In [ ]:
# The following two plots provide a graphical overview of the correlations between the individual features.
# The following two plots are derived from the following page:https://www.geeksforgeeks.org/exploring-correlation-in-python/

In [ ]:
corrs = train_ex_filled.corr() 
  
cg = sns.clustermap(corrs, cmap ="YlGnBu", linewidths = 0.1); 
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation = 0);
cg

# 7. Identify the promising transformations you may want to apply.

The following transformations seem to be reasonable and therfore promising:
* powerPS+yearOfRegistration
* powerPS+model
* powerPS+name
* powerPS+kilometer
* powerPS+postalCode
* kilometer+model
* model+name
* powerPS+yearOfRegistration+model
* powerPS+yearOfRegistration+name
* powerPS+model+name

# 8. Document what you have learned.
The difference between the raw data and the adjusted data is huge. In all features some data had to be deleted to get a plausible result. For the feature "powerPS" the discrepancy was the biggest. We had values from 0 to about 2000ps. The cleaned up data covers a range of 10-500ps. This caused some data to be lost, but the plausible values without outliers allowed us to make more accurate predictions. 
Since the highest category is 150000 in the system, it is difficult to understand and edit this feature accurately. 
Furthermore we could not consider the data in the category 5000km as plausible. Because this category had the lowest average price, which didn't make much sense for us. 
This feature overstrained us in the data preparation. 





# 9. Prepare the Data
In the following subchapters the data is prepared in a way, that the algorithms can afterwards handle the data. For these parts the findings of the data exploratory part are used.

## 9.1 Data cleaning, feature selection

The outliers of each features are removed, the missing values filled and attributes which do not provide useful informations for the task are dropped. To not manipulate the original dataset it is worked on a copy of it.

In [ ]:
train_clean = train.copy()
train_clean.shape

In [ ]:
# Applying the framing which excludes the unrealistic values of each rows

def frame_df(df):
    '''
    Function applies the framing which excludes the unrealistic values of the dataset by dropping the 
    corresponding rows. Should be used just with the train dataset. If applied to the test dataset
    rows will be missing.
    '''
    df = df.loc[(df.price < 50000)]
    df = df.loc[(df.price > 10)]
    df = df.loc[(df.powerPS < 500)]
    df = df.loc[(df.powerPS > 10)]
    df = df.loc[(df.kilometer < 150000)]
    df = df.loc[(df.kilometer > 5000)]
    df = df.loc[(df.yearOfRegistration < 2016)]
    df = df.loc[(df.yearOfRegistration > 1995)]
    df = df.loc[(df.monthOfRegistration > 0)]
    df = df.loc[(df.postalCode < 98000)]
    df = df.loc[(df.postalCode > 1000)]
    
    print(df.shape)    
    print('Compared to the original dataset {} rows or {:.2f}% are dropped'
          .format(
          (train.shape[0]-df.shape[0]),
          (100-df.shape[0]/train.shape[0]*100)))
    
    return df

In [ ]:
train_framed = frame_df(train_clean)
train_framed.price.describe()

In [ ]:
# Looking at the most expensive cars after framing it.
train_framed[train_framed.price > 49998]

In [ ]:
# the following funcion applies the findings of the data exploratory part.

def prepare_df(df):
    '''
    Function cleans df corresponding to the findings of the data exploratory part.
    '''
    # dropping cols
    cols_drop = ['dateCrawled',
                 'abtest',
                 'seller',
                 'offerType',
                 'monthOfRegistration',
                 'nrOfPictures'
                ]
    
    df = df.drop(columns = cols_drop)
    
    # OneHotEncode the low cardinality features
    ## features to OneHotEncode
    cols_ohe = ['notRepairedDamage',
                'fuelType',
                'gearbox',
                'vehicleType',
                'brand']
    
    ## OneHotEncoded df
    df_encoded = pd.get_dummies(df[cols_ohe])
    
    ## Joining the orginal input dataset and the encoded one
    df = pd.concat([df, df_encoded], axis=1) 
    
    # Extracting dateparts into more useful parts
    add_datepart(df, 'dateCreated')
    add_datepart(df, 'lastSeen')
    
    # Converting strings to categories
    convert_strings_to_categories(df)
    
    # Converting categories to numbers
    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    
    # Appliying the mean instead of the median to the missing price values with 5000 kilometers    
    mean_low_k = df.kilometer.mean()
    df.kilometer.fillna(mean_low_k, inplace=True)
    
    # Filling rest of NaN's with median
    median_df = df.median()
    df.fillna(median_df, inplace=True)
    
    ## Dropping the encoded features
    df = df.drop(columns = cols_ohe)  

    df_cleaned = df
    
    return df_cleaned

In [ ]:
train_prepared = prepare_df(train_framed);
train_prepared.shape

In [ ]:
#check the nan values
train_prepared.isnull().any().sum()

In [ ]:
# Looking at some random samples
train_prepared.sample(5)

## 9.2 Feature engineering, where appropriate

The feature engineering following points are:

* Discretize continuous features.
* Decompose features (e.g. categorical, date/time, etc.).
* Add promising transformations of features (e.g. log(x), squrt(x), x^2, etc.).
* Aggregate features into promising new features.

In [ ]:
# Aggregating features into new features which are thought to be helpful.

def aggregating_features(df):
    '''
    Function aggregates features based on the findings before.
    '''
    
    df['powerPS+yearOfRegistration'] = df.powerPS + df.yearOfRegistration
    df['powerPS+model'] = df.powerPS + df.model
    df['powerPS+name'] = df.powerPS + df.name
    df['powerPS+kilometer'] = df.powerPS + df.kilometer
    df['powerPS+postalCode'] = df.powerPS + df.postalCode
    df['kilometer+model'] = df.kilometer + df.model
    df['model+name'] = df.model + df.name
    df['powerPS+yearOfRegistration+model'] = df.powerPS + df.yearOfRegistration + df.model
    df['powerPS+yearOfRegistration+name'] = df.powerPS + df.yearOfRegistration + df.name
    df['powerPS+model+name'] = df.powerPS + df.model + df.name
    
    aggregated_df = df
    
    return aggregated_df

In [ ]:
train_aggregated = aggregating_features(train_prepared)
train_aggregated.shape

# 10. Short-List Promising Models
In this part are different models trained and evaluated. Also the individual parts are contained here.

## 10.1 Training many quick and dirty models from different categories and comparing their performance

In [ ]:
train_models = train_aggregated.copy()

In [ ]:
X = train_models.drop('price', axis=1)
y = train_models.price
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=69)
print('Train, validation split:')
print(f'{len(X_train)} train rows\n{len(X_valid)} validation rows\n')

In [ ]:
# Created baseline model function to compare the feature engineering development
def baseline_model_score2(df):
    '''
    Baseline model of simple RandomForestRegressor to measure the impact of the feature engineering steps
    '''
    model = RandomForestRegressor()
    model = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=69)
    model.fit(X_train, y_train)
    print('Metrics:')
    print_scores(model)

In [ ]:
baseline_model_score2(train_models)

In [ ]:
# Comparing the RMSE of the following untuned models to find the best model for the problem
regressors1 = [
    RandomForestRegressor(),
    ExtraTreesRegressor(),
    AdaBoostRegressor(),
    GradientBoostingRegressor(),
    LGBMRegressor(),
    BaggingRegressor(),
    IsolationForest(),
    DecisionTreeRegressor(),
    HistGradientBoostingRegressor(),
    XGBRegressor(),
    ElasticNet()
    ]

log_cols = ["Regressor", "RMSE"]
log 	 = pd.DataFrame(columns=log_cols)

In [ ]:
for rgrs in regressors1:
    name = rgrs.__class__.__name__
    rgrs.fit(X_train, y_train)
    train_predictions = rgrs.predict(X_valid)    
    RMSE = rmse(y_valid, train_predictions)
    log = log.append({'Regressor':name, 'RMSE':RMSE}, ignore_index=True)

plt.title('Regressor RMSE')
plt.xlabel('RMSE')
sns.barplot(x='RMSE', y='Regressor', data=log.sort_values(ascending=True, by='RMSE'), color='b')

In this part just the models with the best 5 models are taken into account. (Adopted part of https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

---
### RandomForestRegressor

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('best_random_rf', 'rb')
# best_random_rf = pickle.load(pickle_in)

In [ ]:
# For the best hyperparameters is looked for with the RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 200, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Creating the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# First creating the base model to tune

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 3,
                               verbose=5,
                               random_state=69, 
                               n_jobs = -1)
# Fitting the random search model
%time rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
print_scores(best_random_rf)

In [ ]:
best_random_rf = rf_random.best_estimator_
%time random_rf_score = print_scores(best_random_rf)

In [ ]:
pickle_out = open('best_random_rf', 'wb')
pickle.dump(best_random_rf, pickle_out)
pickle_out.close()

### Looking with randomized grid search for some models for best tuning parameters and comparing the performance of VotingRegressor to StackingRegressor - Lappert Michael

Besides to the RandomForestRegressor to the 4 best models of the model evaluation part a hyperparamter-tuning is applied.
#### ExtraTreeRegressor

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
pickle_in = open('best_random_et', 'rb')
best_random_et = pickle.load(pickle_in)

In [ ]:
# For the best hyperparameters is looked for with the RandomizedSearchCV

# Number of trees in forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Creating the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# First creating the base model to tune

et = ExtraTreesRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
et_random = RandomizedSearchCV(estimator = et, 
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 3,
                               verbose=5,
                               random_state=69, 
                               n_jobs = -1)
# Fitting the random search model
%time et_random.fit(X_train, y_train)

In [ ]:
et_random.best_params_

In [ ]:
best_random_et = et_random.best_estimator_
%time random_et_score = print_scores(best_random_et)

In [ ]:
pickle_out = open('best_random_et', 'wb')
pickle.dump(best_random_et, pickle_out)
pickle_out.close()

#### XGBRegressor

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('best_random_xgb', 'rb')
# best_random_xgb = pickle.load(pickle_in)

In [ ]:
# For the best hyperparameters is looked for with the RandomizedSearchCV


nthread = [3, 4, 5]
objective = ['reg:squaredlogerror']
learning_rate = [0.01, 0.03, 0.05, 0.07, 0.1]
max_depth = [1, 3, 5, 7]
min_child_weight = [2, 4, 6]
silent = [1]
subsample = [0.6, 0.7, 0.8, 0.9]
colsample_bytree = [0.5, 0.7, 0.9]
n_estimators = [1000]
    


# Creating the random grid
random_grid = {'nthread':nthread,
              'objective':objective,
              'learning_rate':learning_rate,
              'max_depth':max_depth,
              'min_child_weight':min_child_weight,
              'silent':silent,
              'subsample':subsample,
              'colsample_bytree':colsample_bytree,
              'n_estimators':n_estimators}

In [ ]:
# First creating the base model to tune

xgb = XGBRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb, 
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 3,
                               verbose=5,
                               random_state=69, 
                               n_jobs = -1)
# Fitting the random search model
%time xgb_random.fit(X_train, y_train)

In [ ]:
xgb_random.best_params_

In [ ]:
best_random_xgb = xgb_random.best_estimator_
%time random_xgb_score = print_scores(best_random_xgb)

In [ ]:
pickle_out = open('best_random_xgb', 'wb')
pickle.dump(best_random_xgb, pickle_out)
pickle_out.close()

#### HistGradiantBoostingRegressor

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('best_random_hgb', 'rb')
# best_random_hgb = pickle.load(pickle_in)

In [ ]:
# For the best hyperparameters is looked for with the RandomizedSearchCV


learning_rate = [0.05, 0.1, 0.01]
max_iter = [1000, 1500, 2000]
max_leaf_nodes = [2, 4, 8, 16, 32, 64, 128]
max_depth = [25, 50, 75]
min_samples_leaf = [5, 10, 20, 30, 35]
scoring = ['reg:squaredlogerror']
l2_regularization = [1.5]
#n_estimators = [1000]
    


# Creating the random grid
random_grid = {'learning_rate':learning_rate,
              'max_iter':max_iter,
              'max_leaf_nodes':max_leaf_nodes,
              'max_depth':max_depth,
              'min_samples_leaf':min_samples_leaf,
              'scoring':scoring,
               'l2_regularization':l2_regularization
              }

In [ ]:
# First creating the base model to tune

hgb =  HistGradientBoostingRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across different combinations, and use all available cores
hgb_random = RandomizedSearchCV(estimator = hgb, 
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 3,
                               verbose=5,
                               random_state=69, 
                               n_jobs = -1)
# Fitting the random search model
%time hgb_random.fit(X_train, y_train)

In [ ]:
hgb_random.best_params_

In [ ]:
best_random_hgb = hgb_random.best_estimator_
%time random_hgb_score = print_scores(best_random_hgb)

In [ ]:
pickle_out = open('best_random_hgb', 'wb')
pickle.dump(best_random_hgb, pickle_out)
pickle_out.close()

#### LGBMRegressor

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('best_random_lgbm', 'rb')
# best_random_lgbm = pickle.load(pickle_in)

In [ ]:
# For the best hyperparameters is looked for with the RandomizedSearchCV

num_leaves =list(range(8, 120, 4))
min_data_in_leaf = [5, 10, 20, 40, 60, 100]
max_depth = [3, 4, 5, 6, 8, 12, 16, -1]
learning_rate = [0.2, 0.15, 0.1, 0.05, 0.01, 0.005]
bagging_freq = [1, 2, 3, 4, 5, 6, 7]
bagging_fraction = np.linspace(0.3, 0.95, 10)
reg_alpha = np.linspace(0.1, 0.95, 10)
reg_lambda = np.linspace(0.1, 0.95, 10)



random_grid = {'num_leaves': num_leaves,
               'min_data_in_leaf': min_data_in_leaf,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'bagging_freq': bagging_freq,
               'bagging_fraction': bagging_fraction,
               'reg_alpha': reg_alpha,
               'reg_lambda': reg_lambda
              }


In [ ]:
# First creating the base model to tune

lgbm =  LGBMRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across different combinations, and use all available cores
lgbm_random = RandomizedSearchCV(estimator = lgbm, 
                                 param_distributions = random_grid,
                                 n_iter = 30,
                                 cv = 3,
                                 verbose=5,
                                 random_state=69, 
                                 n_jobs = -1)
# Fitting the random search model
%time lgbm_random.fit(X_train, y_train)

In [ ]:
lgbm_random.best_params_

In [ ]:
best_random_lgbm = lgbm_random.best_estimator_
%time random_lgbm_score = print_scores(best_random_lgbm)

In [ ]:
pickle_out = open('best_random_lgbm', 'wb')
pickle.dump(best_random_lgbm, pickle_out)
pickle_out.close()

#### VotingRegressor
Here are the models with the highest RMSE score in the randomized grid search part ensembled. This is done with the VotingRegressor of sklearn. The VotingRegressor combines the given machine learning regressors and returns the average predicted value. This can be useful to balance out the individual weaknesses of otherwise eaqually well performing models.

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('VotingRegressor', 'rb')
# votreg = pickle.load(pickle_in)

In [ ]:
# https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier
from sklearn.ensemble import VotingRegressor

train_models = train_aggregated.copy()

X = train_models.drop('price', axis=1)
y = train_models.price
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=69)
print('Train, validation split:')
print(f'{len(X_train)} train rows\n{len(X_valid)} validation rows\n')

In [ ]:
votreg = VotingRegressor(estimators=[('lgbm', best_random_lgbm),
                                     ('hgb', best_random_hgb),
                                     ('et', best_random_et)])

%time votreg = votreg.fit(X_train, y_train)

In [ ]:
print_scores(votreg)

In [ ]:
pickle_out = open('VotingRegressor', 'wb')
pickle.dump(votreg, pickle_out)
pickle_out.close()

Each of the VotingRegressors estimators prediction as well as the VotingRegressors prediction itself are visualized.

In [ ]:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_regressor.html#sphx-glr-auto-examples-ensemble-plot-voting-regressor-py
xt = X_train[:20]

pred1 = best_random_lgbm.predict(xt)
pred2 = best_random_hgb.predict(xt)
pred3 = best_random_et.predict(xt)
pred4 = votreg.predict(xt)

plt.figure();
plt.figure(figsize=(15,8));
plt.plot(pred1, 'gd', label='LGBMRegressor');
plt.plot(pred2, 'b^', label='HistGradientBoostinRegressor');
plt.plot(pred3, 'ys', label='ExtraTreesRegressor');
plt.plot(pred4, 'r*', ms=10, label='VotingRegressor');

plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False);
plt.ylabel('predicted', fontsize=15);
plt.xlabel('training samples', fontsize = 15);
plt.legend(loc="best", fontsize=15);
plt.title('Regressor predictions and their average', fontsize = 20);

plt.show();

#### StackingRegressor
In the following cells is the performance of the VotingRegressor compared to the StackingRegressor. With the stecked generalization, which is perfomed with the StackingRegressor, the idea is to reduce the biases of the different models. This is done by stacking the predictions of each individual estimator and used as input to a final estimator.

In [ ]:
# Uncomment the below code and skip next 2 cells in case the noteboook ran already once on this computer.
# pickle_in = open('StackingRegressor', 'rb')
# stackreg = pickle.load(pickle_in)

In [ ]:
# https://scikit-learn.org/stable/modules/ensemble.html#stacked-generalization
from sklearn.ensemble import StackingRegressor

estimators = [('lgbm', best_random_lgbm),
              ('hgb', best_random_hgb),
              ('et', best_random_et)]

stackreg = StackingRegressor(
    estimators=estimators,
    final_estimator=best_random_rf)

In [ ]:
%time stackreg.fit(X_train, y_train)

In [ ]:
print_scores(stackreg)

In [ ]:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_regressor.html#sphx-glr-auto-examples-ensemble-plot-voting-regressor-py
xt = X_train[:20]

pred1 = best_random_lgbm.predict(xt)
pred2 = best_random_hgb.predict(xt)
pred3 = best_random_et.predict(xt)
pred4 = best_random_rf.predict(xt)
pred5 = stackreg.predict(xt)

plt.figure();
plt.figure(figsize=(15,8));
plt.plot(pred1, 'gd', label='LGBMRegressor');
plt.plot(pred2, 'b^', label='HistGradientBoostinRegressor');
plt.plot(pred3, 'ys', label='ExtraTreesRegressor');
plt.plot(pred4, 'co', label='RandomforestRegressor');
plt.plot(pred4, 'r*', ms=10, label='StackingRegressor');


plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False);
plt.ylabel('predicted', fontsize=15);
plt.xlabel('training samples', fontsize = 15);
plt.legend(loc="best", fontsize=15);
plt.title('Regressor predictions and their average', fontsize = 20);

plt.show();

In [ ]:
pickle_out = open('StackingRegressor', 'wb')
pickle.dump(stackreg, pickle_out)
pickle_out.close()

---
### Linear Regression Model - Bischof Tobias

In [ ]:
#The following model is based on the structure of the following two pages:https://towardsdatascience.com/a-beginners-guide-to-linear-regression-in-python-with-scikit-learn-83a8f7ae2b4f
                                                                        #:https://towardsdatascience.com/introduction-to-linear-regression-141cde7a46b2

In [ ]:
#Make a copy of the data

In [ ]:
train_LR = train_models.copy()

In [ ]:
#Here the correlation of the different features is shown. Since powerPS has the highest correlation, 
#I decided to use this feature to create a LR model.

In [ ]:
corrs = train_LR.corr()
corrs['price'].sort_values(ascending=False).sum(), corrs['price'].sort_values(ascending=False)

In [ ]:
#This plot shows us that the raw data is useless for a prediction based on the feature powerPS and price.

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.distplot(train_LR['powerPS'])

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.distplot(train_LR['price'])

In [ ]:
#Graphical representation of the two features with cleaned values.

In [ ]:
train_LR.plot(x='price', y='powerPS', style='o')  
plt.title('price vs powerPS')  
plt.xlabel('price')  
plt.ylabel('powerPS')  
plt.show()

In [ ]:
#The next step is to assign both features to an axis.

In [ ]:
X = train_LR['powerPS'].values.reshape(-1,1)
y = train_LR['price'].values.reshape(-1,1)

In [ ]:
#The data is divided into a training set and a test set in a ratio of 80 to 20.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#After we have divided the data into training and test sets, the algorithm is trained. To do this, we have to import the class LinearRegression, instantiate it and call the method fit() together with our training data.

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

In [ ]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#Here you can see the value of the intercept and slope calculated by the linear regression algorithm for our data set.

In [ ]:
#To retrieve the intercept:
print(regressor.intercept_)
#For retrieving the slope:
print(regressor.coef_)

In [ ]:
#Since the algorithm is trained, a prediction is now made. For this purpose we will use our test data and see how accurately our algorithm predicts the percentage result.

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
#In the next two plots the predicted values are compared with the actual values.

In [ ]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df

In [ ]:
df1 = df.head(50)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
#Plot with a trend line with the test data 

In [ ]:
plt.scatter(X_test, y_test,  color='gray')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#The following result, cannot make such an accurate prediction. Because the model is based on only one feature(powerPS). 
#A multiple LR would certainly be more accurate since the prediction is based on more than one feature. 

#### Multiple Linear Regression

In [ ]:
#Instead of only one numerical feature, additional features with a high correlation were added. 

In [ ]:
#The data is also divided into "attributes" and "labels". The X-variable contains the 3 features and the y-variable contains labels (price).

In [ ]:
X = train_LR[['kilometer', 'yearOfRegistration', 'powerPS']].values
y = train_LR['price'].values

In [ ]:
#Next we create an overview of the average prices

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
sns.distplot(train_LR['price'])

In [ ]:
#Then we split 80% of the data for the training set, while 20% of the data for the test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Train model

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

In [ ]:
#Prediction based on test data

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
#the difference between the actual value and the predicted value is shown

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1 = df.head(50)

In [ ]:
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
#The additional features brought better results. However, negative price predictions are unrealistic!

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
#Here we see a significant improvement to the model with only one feature. 
#Because the algorithm now combines several values and thus gets more information and the predictions are also more accurate.

---
### MultiLayerPerception - Vogt Tobias

The following model is based on the structure of the following two pages: https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn -> but MLPClassifier changend to MLPRegressor 
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [ ]:
cols_keep = ['price', 'powerPS', 'kilometer', 'yearOfRegistration']
train_M = train_models[cols_keep]
train_M.shape

In [ ]:
df =  train_M
print(df.shape)
df.describe().transpose()

In [ ]:
target_column = ['price'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

In [ ]:
X = df[predictors].values
y = df[target_column].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=40)
print(f'{len(X_train)} train rows + {len(X_valid)} valid rows')

In [ ]:
from sklearn.neural_network import MLPRegressor
regr =  MLPRegressor(hidden_layer_sizes=(100,100), alpha=0.001, learning_rate_init=0.01,
                     power_t=0.5, max_iter=500, shuffle=True,
                     random_state=None, tol=0.0001, verbose=False, warm_start=False,
                     validation_fraction=0.1).fit(X_train, y_train)

In [ ]:
regr.fit(X_train,y_train)

In [ ]:
print_scores(regr)

# 11. Analyzing the most significant variables for RandomForestRegressor

In [ ]:
# https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

def drop_col_feat_imp(model, X_train, y_train, random_state = 69):
    
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = imp_df(X_train.columns, importances)
    return importances_df

In [ ]:
%time drop_col_feat_imp(best_random_rf, X_train, y_train, random_state=69)

# 12. Analze the types of errors the models make.

* What data would a human have used to avoid these errors?

In [ ]:
def plot_prediction_error(fitted_model, X, y):
    """
    A utility function to visualise the prediction errors of regression models.
    
    Args:
        fitted_model: A scikit-learn regression model.
        X: The feature matrix to generate predictions on.
        y: The target vector compare the predictions against.
    """
    y_pred = model.predict(X)
    plt.figure(figsize=(8, 4))
    sns.scatterplot(y, y_pred)
    sns.lineplot([y.min(), y.max()], [y.min(), y.max()], lw=2, color="r")
    plt.xlabel("Actual Median Car Price")
    plt.ylabel("Predicted Median Car Price")
    plt.title(f"Prediction Error for {model.__class__.__name__}")
    plt.show()

In [ ]:
plot_prediction_error(best_random_rf, X_valid, y_valid)

In [ ]:
def plot_residuals(fitted_model, X, y):
    '''
    A utility function to visualise the residuals of regression models.
    
    Args:
        fitted_model: A scikit-learn regression model.
        X: The feature matrix to generate predictions on.
        y: The target vector compare the predictions against.   
    '''
    y_pred = model.predict(X)

    sns.residplot(y_pred, y - y_pred)
    plt.ylabel('Residuals')
    plt.xlabel('Predicted Median Car Price')
    plt.title(f'Residuals for {model.__class__.__name__}')
    plt.show()

In [ ]:
plot_residuals(best_random_rf, X_valid, y_valid)

# 12. Model interpretability Random Forest

In [ ]:
data = train_models

In [ ]:
preds = np.stack([t.predict(X_valid) for t in best_random_rf.estimators_])
# calculate mean and standard deviation for single observation
np.mean(preds[:,0]), np.std(preds[:,0])

In [ ]:
sns.countplot(y='kilometer', data=train_models);

In [ ]:
# prepare figure 
fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, figsize=(12,7), sharex=True)

# plot ground truth
train_models.plot('kilometer', 'price', 'barh', ax=ax0)
# put legend outside plot
ax0.legend(loc='upper left', bbox_to_anchor=(1.0, 0.5))

# plot preds
train_models.plot('kilometer', 'preds_mean', 'barh', xerr='preds_std', alpha=0.6, ax=ax1)
# put legend outside plot
ax1.legend(loc='upper left', bbox_to_anchor=(1.0, 0.5))

plt.show()

# 13. Once you are confident about your final model, measure its performance on the test set to estimate the generalization error.

In [ ]:
test_sub = test.copy()
test_sub.shape

In [ ]:
test_prepared = prepare_df(test_sub)
test_aggregated = aggregating_features(test_prepared)

In [ ]:
preds = best_random_lgbm.predict(test_aggregated)

In [ ]:
sample_submission.shape, preds.shape

In [ ]:
submission = sample_submission['Id'].to_frame()
submission['Predicted'] = preds

In [ ]:
submission.head()

In [ ]:
submission.to_csv('your_path', index=False)

# 14. Solution presentation

## 14.1 Documentation of what is done

### Get the data; 
* here from Kaggle
### Explore the data; 
* looking randomly some data
* searching the missing values
* exploring the number of unique labels of the object features
* a lot of plots like number of cars by brand
### Prepare the Data;
* applying the framing which excludes the unrealistic values of each rows
* dropping cols
* appliying the mean instead of the median to the missing price values with 500 kilometers   
* filling the NaN’s with median
* aggregating features into new features which are thought to be helpful
### Evaluate models
* evaluating performances of many different models such as RandomForestRegressor, LGBMRegressor...
* fine-tuning of different models in individual parts
* combining best scoring models from the evaluation
* looking at the prediction errors the RandomForestRegressor makes
* interpreting the results of the RandomForestRegressor
* predicting as last model step the test data and submitted it several times to the kaggle competition

## 14.2 Reflection of notebook

In [ ]:
train_presentation = train_aggregated.copy()

In [ ]:
#The number of PS in relation to the price
train_presentation = train_presentation.loc[(train_presentation.powerPS < 1000)]
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_presentation, 'powerPS');

In [ ]:
#The year of registration in realtion to the price
train_presentation = train_presentation.loc[(train_presentation.yearOfRegistration<2016)]
train_presentation = train_presentation.loc[(train_presentation.yearOfRegistration>1930)]
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_presentation, 'yearOfRegistration');

In [ ]:
#The number of kilometers in realtion to the price
figsize(8, 6)
plt.rcParams['font.size'] = 15
plot_histogram(train_presentation, 'kilometer');

## 14.3 Explanation of achieving the business objective

The business objective was to build a machine learning model, which predicts the selling price of a car on basis of a given dataset. It can be said, that this objective is achieved. The created model achieves an Roor mean squared error of around 4000 on the test data.
The model could be improved with the following points:

**Feature engineering**
* The whole data set should be normalized.
* The name should be extracted and analyzed through Natural language processing (nlp).
* The evaluation "median price" / "kilometer" does not yet make complete sense. Because the fewer kilometers a car has, the higher the price should be. However, the price is lowest for vehicles with 5000 kilometres. Therefore we conclude that we have not yet fully decomposed this feature to understand all the details.
* It is not found the meaning of every feature as e.g. abtest. For deeper understanding of the dataset it would be helpful to fully understand what data one is working with.

**ML Model**
* It should be taken care of that the model does not overfit the training data
* The random grid search could be extended to more iterations to find possible better combinations
* Instead of a random grid search a 'non randomized' grid search could be applied to not miss any useful combination
* Additional feature engineering rounds could be done after the best model was selected

Besides the mentioned aspects, we are satisfied with our work. We believe that we were able to apply the theoretical contents of the lessons in this practical example and can therefore look back on a successful and exciting project. 
